In [1]:
import numpy as np
import gymnasium as gym
import ale_py
gym.register_envs(ale_py)
import ray
print("ray 版本:", ray.__version__)

ray 版本: 2.40.0


# 学习者

In [5]:
from ray.rllib.algorithms.ppo.ppo import PPOConfig  

env = gym.make("CartPole-v1")

# Create an AlgorithmConfig object from which we can build the
# Learner.
config = (
    PPOConfig()
)
# Construct a new Learner using our config object.
learner = config.build_learner(env=env)

# Needs to be called on the learner before calling any functions.
learner.build()

2025-01-09 15:21:57,235	WARNING rl_module.py:427 -- Could not create a Catalog object for your RLModule! If you are not using the new API stack yet, make sure to switch it off in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. Some algos already use the new stack by default. Ignore this message, if your RLModule does not use a Catalog to build its sub-components.
2025-01-09 15:21:57,236	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!


In [4]:
# # This is a blocking update (given a training batch).
# result = learner.update_from_batch(batch=DUMMY_BATCH, timesteps=TIMESTEPS)

In [ ]:
# 相关 api
# 'before_gradient_based_update',
# 'after_gradient_based_update',# 梯度更新完成，可以分发给学习设备
# 'learner.update_from_batch',
# 'apply_gradients', # 可用于参数服务器梯度更新
# 'compute_gradients',# 每个学习设备可获取梯度，发送给参数服务器
# 'postprocess_gradients',
# 'get_state',

In [10]:
from ray.rllib.core import COMPONENT_RL_MODULE

# Only get the RLModule weights (as numpy, not torch/tf).
rl_module_only_state = learner.get_state(components=COMPONENT_RL_MODULE)
rl_module_only_state

{'should_module_be_updated': None,
 'rl_module': {'_multi_rl_module_spec': {'multi_rl_module_class': 'ray.rllib.core.rl_module.multi_rl_module.MultiRLModule',
   'observation_space': None,
   'action_space': None,
   'inference_only': False,
   'model_config': None,
   'rl_module_specs': {'default_policy': {'module_class': 'ray.rllib.algorithms.ppo.torch.ppo_torch_rl_module.PPOTorchRLModule',
     'observation_space': {'space': 'box',
      'low': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RomOpo6CrUKFACuWTNnHmBgaPgfUH5tH4gGAEwPI0A=',
      'high': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RomOpo6CrUKFACuWTNnOjAwNNQHlF+zA9EAPg8hQA==',
      'shape': (4,),
      'dtype': '<f4'},
     'action_space': {'space': 'discrete', 'n': 2, 'start': 0},
     'inference_only': False,
     'learner_only': False,
     'model_config': {'vf_share_layers': False},
     'catalog_class': 'ray.rllib.algorithms.ppo.ppo_catalog.PPOCatalog'

In [8]:
# Note that `rl_module_only_state` is now a dict:
# {COMPONENT_RL_MODULE: [RLModule's state]}
learner.module.set_state(rl_module_only_state)

In [ ]:
rl_module_only_state['rl_module']['default_policy']

# 学习组

In [4]:
from ray.rllib.algorithms.ppo.ppo import PPOConfig
env = gym.make("CartPole-v1")

# Create an AlgorithmConfig object from which we can build the
# LearnerGroup.
config = (
    PPOConfig()
    # Number of Learner workers (Ray actors).
    # Use 0 for no actors, only create a local Learner.
    # Use >=1 to create n DDP-style Learner workers (Ray actors).
    .learners(num_learners=2)
)

# Construct a new LearnerGroup using our config object.
learner_group = config.build_learner_group(env=env)
learner_group

In [5]:
learner_group._workers

(_WrappedExecutable pid=4752) [W socket.cpp:697] [c10d] The client socket has failed to connect to [kubernetes.docker.internal]:6511 (system error: 10049 - �����������У�������ĵ�ַ��Ч��).
(_WrappedExecutable pid=4752) 2025-01-09 11:45:57,620	WARNING rl_module.py:427 -- Could not create a Catalog object for your RLModule! If you are not using the new API stack yet, make sure to switch it off in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. Some algos already use the new stack by default. Ignore this message, if your RLModule does not use a Catalog to build its sub-components.
(_WrappedExecutable pid=4752) 2025-01-09 11:45:57,635	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(_WrappedExecutable pid=13404) 

[Actor(create_executable_class.<locals>._WrappedExecutable, e92ec8879730b4b45a70e54501000000),
 Actor(create_executable_class.<locals>._WrappedExecutable, c8b7b26a721cda4e17f0cd6701000000)]

In [6]:
# Only get the RLModule weights.
weights = learner_group.get_weights()
learner_group.set_weights(weights)
weights

{'default_policy': OrderedDict([('encoder.actor_encoder.net.mlp.0.weight',
               array([[-0.02326447,  0.12450731, -0.11812747,  0.21217495],
                      [ 0.4346848 , -0.48375547, -0.21687567,  0.25240654],
                      [-0.34222895,  0.11787081, -0.24042445, -0.229842  ],
                      ...,
                      [-0.15495163, -0.29349643, -0.20736992,  0.23380512],
                      [-0.3801734 ,  0.01548886,  0.39030933, -0.15642709],
                      [ 0.43247205,  0.49843848,  0.33006787,  0.19835019]],
                     dtype=float32)),
              ('encoder.actor_encoder.net.mlp.0.bias',
               array([-0.2195344 ,  0.1336295 , -0.01462734, -0.26066726,  0.45592934,
                      -0.20936161,  0.37022448, -0.43611532,  0.47510362, -0.41864234,
                       0.3666327 , -0.09409815, -0.45791686,  0.00248075,  0.22895849,
                      -0.3947724 ,  0.44219404, -0.2898662 ,  0.18256342,  0.24858141,


# 源码 debug

In [2]:
import copy
from ray.rllib.core import COMPONENT_RL_MODULE
from ray.rllib.algorithms.ppo.ppo import PPOConfig
from ray.rllib.algorithms.ppo.torch.ppo_torch_learner import PPOTorchLearner
from ray.rllib.core.learner.learner_group import LearnerGroup
from ray.rllib.utils.deprecation import (
    Deprecated,
    DEPRECATED_VALUE,
    deprecation_warning,
)
from typing import Dict, Any

# 'after_gradient_based_update',# 梯度更新完成，可以分发给学习设备
# 'apply_gradients', # 可用于参数服务器梯度更新 / 每个学习设备可获取梯度(分布式同步后的梯度)，发送给参数服务器
class DebugLearner(PPOTorchLearner):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print("DebugLearner __init__")

    def after_gradient_based_update(
        self,
        *,
        timesteps: Dict[str, Any],
    ) -> None:
        print("DebugLearner after_gradient_based_update")
        return super().after_gradient_based_update(timesteps=timesteps)

    def apply_gradients(self, gradients_dict) -> None:
        print("DebugLearner apply_gradients")
        return super().apply_gradients(gradients_dict)

    def compute_gradients(self, loss_per_module, **kwargs):
        print("DebugLearner compute_gradients")
        return super().compute_gradients(loss_per_module, **kwargs)

config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .environment("CartPole-v1")
    .env_runners(num_env_runners=0)
    .learners(    
        # 一个本地学习者
        # 调试函数:
        # 'after_gradient_based_update',# 梯度更新完成，可以分发给学习设备
        # 'apply_gradients', # 可用于参数服务器梯度更新
        # 'compute_gradients',# 每个学习设备可获取梯度，发送给参数服务器 
        num_learners=0,

        # # 学习组
        # # 调试函数:
        # # 'update_from_batch',
        # num_learners=2,
        # num_gpus_per_learner=0,
    )
    .training(
        learner_class=DebugLearner,# 调试学习者
    )
)

algo = config.build()
for i in range(5):
    print(f"\nTraining iteration {i+1}/{5}")
    result = algo.train()

2025-01-09 16:21:47,803	WARNING ppo.py:295 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
d:\programs\miniconda3\Lib\site-packages\ray\rllib\algorithms\algorithm.py:569: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
d:\programs\miniconda3\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ig

DebugLearner __init__


2025-01-09 16:21:49,880	WARNING util.py:61 -- Install gputil for GPU system monitoring.



Training iteration 1/5
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner com

2025-01-09 16:22:15,149	INFO worker.py:1821 -- Started a local Ray instance.



Training iteration 2/5
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner compute_gradients
DebugLearner apply_gradients
DebugLearner com